<a href="https://colab.research.google.com/github/Sharkar96/BHDQN/blob/main/BHDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import random as r
import numpy as np
import math as m
from numpy import linalg
from shapely.geometry import Point, Polygon, LineString
from shapely import affinity
np.set_printoptions(precision=2)

In [73]:
def point2nparray(p: Point):
  return np.array([p.x, p.y])

Parameters

In [74]:
katt=1
kstr=8
krep=20
p0=3
ps=8
W1=-10
W2=10
W3=-0.2
alpha=-20
gamma=0.95
ts=0.1

In [75]:
class Obstacle:
  def __init__(self, o: Point, A):
    self.A=A
    self.o=o
    self.obstacle=None
    self.createObstacle(o, A)

  def contains(self, *arg):
    return self.obstacle.contains(arg)

  def createObstacle(self, o: Point, A):
    th=360*r.random()
    rngLength=r.random()*A

    vertex=[]
    vertex.append(Point(o.x-rngLength/2,o.y-(A/rngLength)/2))
    vertex.append(Point(o.x+rngLength/2,o.y-(A/rngLength)/2))
    vertex.append(Point(o.x+rngLength/2,o.y+(A/rngLength)/2))
    vertex.append(Point(o.x-rngLength/2,o.y+(A/rngLength)/2))

    self.obstacle=affinity.rotate(Polygon(vertex),th)


  def __str__(self):
    return str(self.obstacle)

In [76]:
class Robot:
  def __init__(self, x, y):
    self.x=x
    self.y=y

  def move(self, force: np.ndarray):
    #moves: E, NE, N, NW, W, SW, S, SE
    moves=['E', 'NE', 'N', 'NW', 'W', 'SW', 'S', 'SE']
    possibleMoves= np.array([[1,0],
                             [0.707,0.707],
                             [0,1],
                             [-0.707,0.707],
                             [-1,0],
                             [-0.707,-0.707],
                             [0,-1],
                             [0.707,-0.707]]
                            )

    chosenMove=np.argmax(np.dot(force, possibleMoves.T))
    self.x=self.x + ts*possibleMoves[chosenMove][0]*linalg.norm(force)
    self.y=self.y + ts*possibleMoves[chosenMove][1]*linalg.norm(force)
    print(moves[chosenMove])
    return

  def __str__(self):
    return str(Point(self.x,self.y))

In [77]:
class Maze:
  def __init__(self, A):
    self.l=m.sqrt(A)
    self.coordinates=np.array([[0,0],
                              [self.l,0],
                              [self.l,self.l],
                              [0,self.l]])
    self.obstacles=[]
    self.goals=[]
    self.robot=None

#WHAT ABOUT THE NORMALIZATION OF THE POTENTIAL FIELD?
  def gravitationalForce(self):
    robotPosition=np.array([self.robot.x, self.robot.y])
    resultingF=np.array([0, 0])
    for i in self.goals:
      #attractive force
      normalAttF= katt*(robotPosition - point2nparray(i))
      #black hole force
      blackHoleF=np.array([0, 0])
      if linalg.norm(robotPosition-point2nparray(i))<=ps:
        blackHoleF= blackHoleF - kstr*(ps-(robotPosition - point2nparray(i)))

      resultingF= resultingF + normalAttF + blackHoleF
    #TODO: OBSTACLE REPULSIVE FORCE
    return -resultingF

  #check if a goal spawned inside an obstacle
  def isGoalOccluded(self, p: Point):
    for i in self.obstacles:
      if i.contains(p):
        return True
    return False

  #check if the obstacle spawned over a goal
  def isObstacleOccluding(self, p: Polygon):
    for i in self.goals:
      if p.contains(i):
        return True
    return False

  def spawnRobot(self, x=None, y=None):
      if x is not None and y is not None:
        if self.isGoalOccluded(Point(x,y)):
          print('unavailable position')
          return
        else:
          self.robot=Robot(x,y)
      else:
        candidate=Point(r.random()*self.l,r.random()*self.l)
        while self.isGoalOccluded(candidate):
          candidate=Point(r.random()*self.l,r.random()*self.l)
        self.robot=Robot(candidate.x,candidate.y)

  def addGoals(self,n=1):
    for i in range(n):
      candidate=Point(r.random()*self.l,r.random()*self.l)
      while self.isGoalOccluded(candidate):
        candidate=Point(r.random()*self.l,r.random()*self.l)
      self.goals.append(candidate)


  def addObstacles(self, n=1):
    #maximum area is 20% of entire maze
    maxA=0.2
    for i in range(n):
      candidate=Obstacle(o= Point(r.random()*self.l,r.random()*self.l),
                                     A= self.l*self.l*maxA*r.random())
      while self.isObstacleOccluding(candidate):
        candidate=Obstacle(o= Point(r.random()*self.l,r.random()*self.l),
                                     A= self.l*self.l*maxA*r.random())
      self.obstacles.append(candidate)

  def print(self):
    print('coordinates: '+ str(self.coordinates))
    print('robot: ')
    print(self.robot)
    print('goals: ')
    for i in self.goals:
      print(i)
    print('obstacles: ')
    for i in self.obstacles:
      print(i)



In [80]:
maze=Maze(1000)
maze.addGoals(1)

maze.spawnRobot(x=0,y=0)
print(maze.gravitationalForce())
maze.print()
epochs=100
for epoch in range(epochs):
  maze.robot.move(maze.gravitationalForce())


[28.85 30.11]
coordinates: [[ 0.    0.  ]
 [31.62  0.  ]
 [31.62 31.62]
 [ 0.   31.62]]
robot: 
POINT (0 0)
goals: 
POINT (28.854341214826565 30.11052007531234)
obstacles: 
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
SW
SW
NE
SW
